In [1]:
from pyspark.sql import SparkSession  
from pyspark.sql.functions import *
import time



spark = (SparkSession
  .builder
  .appName("AppName")
    .master("spark://spark-master:7077")
  .getOrCreate())

spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
csv_file = "compras.csv"

22/12/03 18:24:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
start_time=time.time()
df = (spark.read.format("csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .option("sep", ";")
    .load(csv_file)).withColumn("data",to_timestamp(regexp_replace('data,,', ',', ''), "MM/dd/yyyy"))
print(f"Execution time: {time.time() - start_time}")

start_time=time.time()
df.createOrReplaceTempView("compras")
print(f"Execution time: {time.time() - start_time}")

Execution time: 4.640423059463501
Execution time: 0.20372915267944336


In [5]:
def measure_time(query):
    start_time = time.time()
    results = spark.sql(query)
    print(f"Execution time: {time.time() - start_time}")
#     return results.show(results.count(), truncate=False)

In [6]:
query="""SELECT nome as nome_cliente, SUM(quantidade) as total_itens FROM compras GROUP BY nome"""
measure_time(query)

Execution time: 0.008726119995117188


In [7]:

query="""SELECT nome as nome_cliente, SUM(quantidade) as total_itens FROM compras GROUP BY nome"""
measure_time(query)


Execution time: 0.007295370101928711


In [8]:
query = """SELECT vendedor_nome, SUM(valor) as total_vendido FROM compras WHERE YEAR(data)=2022 GROUP BY vendedor_nome ORDER BY 2 DESC"""
measure_time(query)

Execution time: 0.04741859436035156


In [9]:
query = """SELECT produto FROM (SELECT produto, SUM(quantidade) FROM compras WHERE YEAR(data) > (2022-5) GROUP BY produto ORDER BY 2 DESC LIMIT 1)"""
measure_time(query)

Execution time: 0.03972673416137695


In [10]:
query = """SELECT produto FROM (SELECT produto, SUM(quantidade) FROM compras WHERE YEAR(data) > (2022-5) GROUP BY produto ORDER BY 2 ASC LIMIT 1)"""
measure_time(query)

Execution time: 0.02541661262512207
